<h1>Chicago Crime Analysis</h1>

<h3>Analyst: Mark Anthony D. Trijo</h3>

<h3>Import Libraries</h3>

In [10]:
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import warnings 
warnings.filterwarnings('ignore')
import os

<h3>Set a Dataframe</h3>

In [11]:
df = pd.read_csv('datasets\\Chicago_Crimes.csv')

In [12]:
df['Updated On'] = pd.to_datetime(df['Updated On'],dayfirst=True,errors = 'coerce')

In [13]:
df['Month'] = df['Updated On'].dt.month

<h3>Import Folium and heatmap</h3>

In [14]:
import folium
from folium.plugins import HeatMap

In [15]:
df['Primary Type'].value_counts()

Primary Type
THEFT                                59201
BATTERY                              44579
CRIMINAL DAMAGE                      27094
ASSAULT                              22615
MOTOR VEHICLE THEFT                  19604
OTHER OFFENSE                        16781
DECEPTIVE PRACTICE                   14991
BURGLARY                              8443
ROBBERY                               8200
WEAPONS VIOLATION                     7164
NARCOTICS                             6627
CRIMINAL TRESPASS                     5045
OFFENSE INVOLVING CHILDREN            1597
CRIMINAL SEXUAL ASSAULT               1573
SEX OFFENSE                           1210
PUBLIC PEACE VIOLATION                1006
INTERFERENCE WITH PUBLIC OFFICER       771
HOMICIDE                               553
STALKING                               520
ARSON                                  452
PROSTITUTION                           287
CONCEALED CARRY LICENSE VIOLATION      249
LIQUOR LAW VIOLATION                   19

In [20]:
df_clean = df.dropna(subset=["Latitude", "Longitude", "Primary Type", "District"])
df_clean['District'] = df_clean['District'].astype(str)
primary_types = df_clean["Primary Type"].unique()


output_dir = "heatmaps"
os.makedirs(output_dir, exist_ok=True)


for crime_type in primary_types:
    df_type = df_clean[df_clean["Primary Type"] == crime_type]
    
    top_district = df_type["District"].value_counts().idxmax()
    df_filtered = df_type[df_type["District"] == top_district]

    heat_data = df_filtered[["Latitude", "Longitude"]].values.tolist()

    base_map = folium.Map(location=[41.8781, -87.6298], zoom_start=11)

    HeatMap(heat_data, radius=10).add_to(base_map)

    
    safe_name = crime_type.replace('/', '_').replace(' ', '_')
    filepath = os.path.join(output_dir, f"heatmap_{safe_name}.html")
    base_map.save(filepath)

In [19]:
master_map = folium.Map(location=[41.8781, -87.6298], zoom_start=11)


for crime_type in primary_types:
    df_type = df_clean[df_clean["Primary Type"] == crime_type]
    top_district = df_type["District"].value_counts().idxmax()
    df_filtered = df_type[df_type["District"] == top_district].dropna(subset=["Latitude", "Longitude"])
    heat_data = df_filtered[["Latitude", "Longitude"]].values.tolist()
    
    layer_name = f"{crime_type} (District {top_district})"
    heat_layer = folium.FeatureGroup(name=layer_name)
    HeatMap(heat_data, radius=10).add_to(heat_layer)
    heat_layer.add_to(master_map)


folium.LayerControl().add_to(master_map)


master_map.save("Chicago_Crimes_Master_Heatmap.html")

In [18]:
master_map